In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import string

from statsmodels.formula.api import ols
from statsmodels.stats.outliers_influence import variance_inflation_factor
import statsmodels.api as sm
import scipy.stats as stats
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import RFE

from sklearn.metrics import r2_score

In [4]:
df = pd.read_csv('master_df.csv')

In [18]:
df['long'] = df['long'].round(2)
df['lat'] = df['lat'].round(2)
df.head()
df2 = df[['price', 'lat', 'long']]
to_dum = ['lat', 'long']

In [19]:
df2[to_dum] = df2[to_dum].apply(pd.Categorical)

In [20]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19054 entries, 0 to 19053
Data columns (total 3 columns):
price    19054 non-null float64
lat      19054 non-null category
long     19054 non-null category
dtypes: category(2), float64(1)
memory usage: 192.4 KB


In [21]:
df2 = pd.get_dummies(df2, drop_first = True)

In [22]:
def col_format(col):
    col = col.replace('.', '_')
    col = col.replace('-', 'neg_')
    return col
df2.columns = [col_format(col) for col in df2.columns]

In [23]:
target = 'price'
x_cols = list(df2.columns)
x_cols.remove('price')

In [24]:
X = df2[x_cols]
vif = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
list(zip(x_cols, vif))

[('lat_47_18', 2.219260488624336),
 ('lat_47_19', 8.317453526483444),
 ('lat_47_2', 10.979810916899122),
 ('lat_47_21', 13.21257743305494),
 ('lat_47_22', 3.222731702998677),
 ('lat_47_23', 2.025317887695681),
 ('lat_47_24', 2.6294731828795),
 ('lat_47_25', 2.8287157532957115),
 ('lat_47_26', 30.573432784460323),
 ('lat_47_27', 17.41097795858822),
 ('lat_47_28', 20.447890917921118),
 ('lat_47_29', 29.76092979993357),
 ('lat_47_3', 34.22435287522712),
 ('lat_47_31', 3.9166091572301616),
 ('lat_47_32', 43.72452311767342),
 ('lat_47_33', 51.61507581467317),
 ('lat_47_34', 47.770818246785495),
 ('lat_47_35', 9.607904254704495),
 ('lat_47_36', 20.527511159397836),
 ('lat_47_37', 26.0508809051385),
 ('lat_47_38', 5.97253753453051),
 ('lat_47_39', 36.82891086453627),
 ('lat_47_4', 33.17589041767454),
 ('lat_47_41', 25.50241562303117),
 ('lat_47_42', 32.80069490539962),
 ('lat_47_43', 35.43996441184915),
 ('lat_47_44', 45.36677659345154),
 ('lat_47_45', 2.6828343248311106),
 ('lat_47_46', 51.4

In [25]:
train, test = train_test_split(df2)

In [26]:
predictors = '+'.join(x_cols)
formula = target + '~' + predictors
model = ols(formula=formula, data=train).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.363
Model:                            OLS   Adj. R-squared:                  0.356
Method:                 Least Squares   F-statistic:                     53.03
Date:                Wed, 03 Jun 2020   Prob (F-statistic):               0.00
Time:                        10:35:45   Log-Likelihood:            -1.9819e+05
No. Observations:               14290   AIC:                         3.967e+05
Df Residuals:                   14137   BIC:                         3.978e+05
Df Model:                         152                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept        5.966e+05   3.16e+05      1.888      0.059   -2.29e+04    1.22e+06
lat_47_18        5.795e+04   2.83e+05      0.205      0.838   -4.96e+05    6.12e+05
lat_47_19       -1.067e+05   2.63e+05     -0.405      0.685   -6.23e+05    4.09e+05
lat_47_2        -9.873e+04   2.62e+05     -0.377      0.706   -6.12e+05    4.14e+05
lat_47_21        -1.35e+05   2.61e+05     -0.517      0.605   -6.47e+05    3.77e+05
lat_47_22       -4.092e+04   2.74e+05     -0.149      0.881   -5.78e+05    4.96e+05
lat_47_23        2.239e+05   2.89e+05      0.776      0.438   -3.42e+05     7.9e+05
lat_47_24        5.867e+04   2.76e+05      0.213      0.832   -4.82e+05    5.99e+05
lat_47_25        1.021e+05   2.79e+05      0.366      0.714   -4.44e+05    6.48e+05
lat_47_26       -9.614e+04   2.59e+05     -0.371      0.711   -6.05e+05    4.12e+05
lat_47_27       -2.752e+04   2.61e+05     -0.106      0.916   -5.38e+05    4.83e+05
lat_47_28       -6.883e+04    2.6e+05     -0.265      0.791   -5.79e+05    4.41e+05
lat_47_29       -1.116e+05    2.6e+05     -0.430      0.667    -6.2e+05    3.97e+05
lat_47_3        -5.842e+04   2.59e+05     -0.225      0.822   -5.67e+05     4.5e+05
lat_47_31       -4.171e+04   2.59e+05     -0.161      0.872   -5.49e+05    4.66e+05
lat_47_32       -3.955e+04   2.59e+05     -0.153      0.879   -5.47e+05    4.68e+05
lat_47_33       -3.965e+04   2.59e+05     -0.153      0.878   -5.47e+05    4.68e+05
lat_47_34        5569.9309   2.59e+05      0.021      0.983   -5.02e+05    5.13e+05
lat_47_35       -1.221e+04   2.59e+05     -0.047      0.962    -5.2e+05    4.95e+05
lat_47_36       -3.553e+04   2.59e+05     -0.137      0.891   -5.43e+05    4.72e+05
lat_47_37       -5.608e+04   2.59e+05     -0.217      0.828   -5.63e+05    4.51e+05
lat_47_38       -4.822e+04   2.59e+05     -0.186      0.852   -5.56e+05    4.59e+05
lat_47_39       -4.093e+04   2.59e+05     -0.158      0.875   -5.49e+05    4.67e+05
lat_47_4        -1.352e+04   2.59e+05     -0.052      0.958   -5.22e+05    4.95e+05
lat_47_41         1.85e+04    2.6e+05      0.071      0.943   -4.91e+05    5.28e+05
lat_47_42       -2.956e+04   2.59e+05     -0.114      0.909   -5.38e+05    4.79e+05
lat_47_43        6367.8860   2.59e+05      0.025      0.980   -5.02e+05    5.15e+05
lat_47_44        6003.6074   2.59e+05      0.023      0.982   -5.02e+05    5.14e+05
lat_47_45       -8230.9503   2.59e+05     -0.032      0.975   -5.16e+05    4.99e+05
lat_47_46        1.138e+04   2.59e+05      0.044      0.965   -4.96e+05    5.19e+05
lat_47_47       -4322.6882   2.59e+05     -0.017      0.987   -5.12e+05    5.03e+05
lat_47_48        1.503e+04   2.59e+05      0.058      0.954   -4.92e+05    5.22e+05
lat_47_49        -3.15e+04   2.59e+05     -0.122      0.903   -5.39e+05    4.76e+05
lat_47_5        -2.076e+04   2.59e+05     -0.080      0.936

In [28]:
df['long'] = df['long'].round(1)
df['lat'] = df['lat'].round(1)
df.head()
df2 = df[['price', 'lat', 'long']]
to_dum = ['lat', 'long']

In [29]:
df2[to_dum] = df2[to_dum].apply(pd.Categorical)

In [30]:
df2 = pd.get_dummies(df2, drop_first = True)

In [31]:
df2.columns = [col_format(col) for col in df2.columns]

In [32]:
x_cols = list(df2.columns)
x_cols.remove('price')

In [33]:
X = df2[x_cols]
vif = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
list(zip(x_cols, vif))

[('lat_47_3', 0.3037335694856885),
 ('lat_47_4', 0.18869852697114683),
 ('lat_47_5', 1.0613476436738645),
 ('lat_47_6', 1.1611918202213467),
 ('lat_47_7', 0.12485520640405649),
 ('lat_47_8', 0.4340979767176175),
 ('long_neg_122_4', 0.7646521106606702),
 ('long_neg_122_3', 0.3515675698375477),
 ('long_neg_122_2', 0.05096820866624393),
 ('long_neg_122_1', 0.6621804503599368),
 ('long_neg_122_0', 0.004190550954104065),
 ('long_neg_121_9', 1.1024573008174197),
 ('long_neg_121_8', 1.9201598916360412),
 ('long_neg_121_7', 1.2593932107855021),
 ('long_neg_121_6', 1.0),
 ('long_neg_121_5', 1.005359234280465),
 ('long_neg_121_4', 1.0410449763192677),
 ('long_neg_121_3', 1.02312133451927)]

In [34]:
train, test = train_test_split(df2)

In [35]:
predictors = '+'.join(x_cols)
formula = target + '~' + predictors
model = ols(formula=formula, data=train).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.256
Model:                            OLS   Adj. R-squared:                  0.255
Method:                 Least Squares   F-statistic:                     288.4
Date:                Wed, 03 Jun 2020   Prob (F-statistic):               0.00
Time:                        10:37:59   Log-Likelihood:            -1.9971e+05
No. Observations:               14290   AIC:                         3.995e+05
Df Residuals:                   14272   BIC:                         3.996e+05
Df Model:                          17                                         
Covariance Type:            nonrobust                                         
==================================================================================
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept       4.127e+05   5.02e+04      8.225      0.000    3.14e+05    5.11e+05
lat_47_3         852.0315   2.64e+04      0.032      0.974   -5.08e+04    5.25e+04
lat_47_4        1.822e+04   2.56e+04      0.711      0.477    -3.2e+04    6.84e+04
lat_47_5        1.209e+05   2.56e+04      4.716      0.000    7.06e+04    1.71e+05
lat_47_6        4.509e+05   2.52e+04     17.873      0.000    4.01e+05       5e+05
lat_47_7         2.94e+05   2.54e+04     11.574      0.000    2.44e+05    3.44e+05
lat_47_8        1.871e+05   2.69e+04      6.945      0.000    1.34e+05     2.4e+05
long_neg_122_4  -1.49e+05    4.4e+04     -3.390      0.001   -2.35e+05   -6.29e+04
long_neg_122_3 -1.611e+05   4.38e+04     -3.677      0.000   -2.47e+05   -7.52e+04
long_neg_122_2 -5.142e+04   4.38e+04     -1.175      0.240   -1.37e+05    3.44e+04
long_neg_122_1 -8.868e+04    4.4e+04     -2.015      0.044   -1.75e+05   -2399.610
long_neg_122_0 -1.058e+05   4.41e+04     -2.400      0.016   -1.92e+05   -1.94e+04
long_neg_121_9 -9.774e+04   4.64e+04     -2.106      0.035   -1.89e+05   -6760.512
long_neg_121_8 -1.094e+05   4.91e+04     -2.226      0.026   -2.06e+05   -1.31e+04
long_neg_121_7 -4.114e+04   6.12e+04     -0.672      0.501   -1.61e+05    7.88e+04
long_neg_121_6 -3.949e-10   3.73e-10     -1.059      0.290   -1.13e-09    3.36e-10
long_neg_121_5 -4.298e+05   2.88e+05     -1.494      0.135   -9.94e+05    1.34e+05
long_neg_121_4 -4.062e+05   1.04e+05     -3.894      0.000   -6.11e+05   -2.02e+05
long_neg_121_3 -5.487e+05   1.49e+05     -3.691      0.000    -8.4e+05   -2.57e+05
==============================================================================
Omnibus:                     9052.574   Durbin-Watson:                   2.016
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           136253.635
Skew:                           2.825   Prob(JB):                         0.00
Kurtosis:                      17.032   Cond. No.                     2.19e+18
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 4.21e-33. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""